# Movie rating database analysis

## Original data source disclaimer

This dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service. It contains 100836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018.

The data are contained in the files `movies.csv`, `ratings.csv` and `tags.csv`. More details about the contents and use of all these files follows.

This GroupLens data set is publicly available for download at <http://grouplens.org/datasets/>.

To acknowledge use of the dataset in publications, please cite the following paper:

> F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

All ratings are contained in the file `ratings.csv`. Each line of this file after the header row represents one rating of one movie by one user, and has the following format:

    userId,movieId,rating,timestamp

Movie information is contained in the file `movies.csv`. Each line of this file after the header row represents one movie, and has the following format:

    movieId,title,genres

All tags are contained in the file `tags.csv`. Each line of this file after the header row represents one tag applied to one movie by one user, and has the following format:

    userId,movieId,tag,timestamp


The features involved in all datasets are the following:
1. **userId** MovieLens users were selected at random for inclusion. Their ids have been anonymized. User ids are consistent between `ratings.csv` and `tags.csv` (i.e., the same id refers to the same user across the two files). All selected users had rated at least 20 movies.
2. **movieId** Only movies with at least one rating or tag are included in the dataset. These movie ids are consistent with those used on the MovieLens web site (e.g., id `1` corresponds to the URL <https://movielens.org/movies/1>). Movie ids are consistent between `ratings.csv`, `tags.csv` and `movies.csv` (i.e., the same id refers to the same movie across all data files).
3. **rating** Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).
4. **timestamp** Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.
5. **tag** Tags are user-generated metadata about movies. Each tag is typically a single word or short phrase. The meaning, value, and purpose of a particular tag is determined by each user.
6. **title** Movie titles are entered manually or imported from <https://www.themoviedb.org/>, and include the year of release in parentheses. Errors and inconsistencies may exist in these titles.
7. **genres** Genres are a pipe-separated list, and are selected from the following: {Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,(no genres listed)}



## PySpark environment setup

In [1]:
import findspark
findspark.init()

In [2]:
findspark.find()
import pyspark
findspark.find()

'/opt/spark-2.4.4-bin-hadoop2.7'

In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

## Functionality imports, data source and Spark DataFrames setup

In [4]:
import pyspark.sql.functions as F
from pyspark.sql.types import TimestampType, DateType, IntegerType
from pyspark.ml.recommendation import ALS

from IPython.display import display, Markdown

Import the csv files into Spark DataFrames:

In [5]:
moviesDF = spark.read\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .csv("movies.csv")

ratingsDF = spark.read\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .csv("ratings.csv")

tagsDF = spark.read\
              .option("header", "true")\
              .option("inferSchema", "true")\
              .csv("tags.csv")

Persist the Spark DataFrames in memory for efficiency purposes, given the various ensuing operations to be applied on them:

In [6]:
moviesDF.cache()
ratingsDF.cache()
tagsDF.cache()

DataFrame[userId: int, movieId: int, tag: string, timestamp: int]

## Datasets metadata analysis
### Head view, size and schema inspection

In [7]:
moviesDF.show(5,False)
print("moviesDF has {} rows".format(moviesDF.count()))
moviesDF.printSchema()

+-------+----------------------------------+-------------------------------------------+
|movieId|title                             |genres                                     |
+-------+----------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                    |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)           |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)          |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)|Comedy                                     |
+-------+----------------------------------+-------------------------------------------+
only showing top 5 rows

moviesDF has 9742 rows
root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [8]:
ratingsDF.show(5)
print("ratingsDF has {} rows".format(ratingsDF.count()))
ratingsDF.printSchema()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows

ratingsDF has 100836 rows
root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [9]:
tagsDF.show(5)
print("tagsDF has {} rows".format(tagsDF.count()))
tagsDF.printSchema()

+------+-------+---------------+----------+
|userId|movieId|            tag| timestamp|
+------+-------+---------------+----------+
|     2|  60756|          funny|1445714994|
|     2|  60756|Highly quotable|1445714996|
|     2|  60756|   will ferrell|1445714992|
|     2|  89774|   Boxing story|1445715207|
|     2|  89774|            MMA|1445715200|
+------+-------+---------------+----------+
only showing top 5 rows

tagsDF has 3683 rows
root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- timestamp: integer (nullable = true)



### Data entities, metrics and dimensions

Given the structural simplicity of the datasets considered, the following can be intuitively inferred:

* **Entities:** Movies (movieID) could be considered as facts
* **Metrics:** Rating, as per the star scale
* **Dimensions:** Users constitute the main dimension, then genres as movie attribute, and tag and timestamp as user attribute (title is just a reader friendly alias for movieId)

## Basic profiling

### Summary of the ratings main statistical descriptors

In [10]:
ratingsDF.select("rating").summary().show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|            100836|
|   mean| 3.501556983616962|
| stddev|1.0425292390606342|
|    min|               0.5|
|    25%|               3.0|
|    50%|               3.5|
|    75%|               4.0|
|    max|               5.0|
+-------+------------------+



### Inspection of missing values 

In [11]:
print("Checking for nulls on columns of moviesDF:")
moviesDF.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in moviesDF.columns]).show()

Checking for nulls on columns of moviesDF:
+-------+-----+------+
|movieId|title|genres|
+-------+-----+------+
|      0|    0|     0|
+-------+-----+------+



In [12]:
print("Checking for nulls on columns of ratingsDF:")
ratingsDF.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in ratingsDF.columns]).show()

Checking for nulls on columns of ratingsDF:
+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     0|      0|     0|        0|
+------+-------+------+---------+



In [13]:
print("Checking for nulls on columns of tagsDF:")
tagsDF.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in tagsDF.columns]).show()

Checking for nulls on columns of tagsDF:
+------+-------+---+---------+
|userId|movieId|tag|timestamp|
+------+-------+---+---------+
|     0|      0|  0|        0|
+------+-------+---+---------+



There are no null values in any of the DataDrames, which is unusual but expected in this case since the data are a well known research benchmark.

### Inspection of distinct values

In [14]:
print("The movies DataFrame has {0} rows, comprising {1} distinct movies and {2} distinct genres"\
      .format(moviesDF.count(),
              moviesDF.select(F.col("movieId")).distinct().count(),
              moviesDF.select(F.col("genres")).distinct().count()))


The movies DataFrame has 9742 rows, comprising 9742 distinct movies and 951 distinct genres


In [15]:
print("The ratings DataFrame has {0} rows, comprising {1} distinct users and {2} distinct movies"\
      .format(ratingsDF.count(),
              ratingsDF.select(F.col("userId")).distinct().count(),
              ratingsDF.select(F.col("movieId")).distinct().count()))


The ratings DataFrame has 100836 rows, comprising 610 distinct users and 9724 distinct movies


Are there any conflicting ratings from the same user for the same movie in ratingsDF?

In [16]:
ratingsDF.select(F.col("userId"),F.col("movieId")).distinct().count()

100836

No, each row is a unique combination of (user, movie)

In [17]:
print("The tags DataFrame has {0} rows, comprising {1} distinct users, {2} distinct movies and {3} distinct tags"\
      .format(tagsDF.count(),
              tagsDF.select(F.col("userId")).distinct().count(),
              tagsDF.select(F.col("movieId")).distinct().count(),
              tagsDF.select(F.col("tag")).distinct().count()))

The tags DataFrame has 3683 rows, comprising 58 distinct users, 1572 distinct movies and 1589 distinct tags


### Casting of timestamp column

In [18]:
ratingsDF = ratingsDF.withColumn("timestamp", ratingsDF["timestamp"].cast(TimestampType()).cast(DateType()))\
                     .withColumnRenamed("timestamp","date")

In [19]:
ratingsDF.show(5)

+------+-------+------+----------+
|userId|movieId|rating|      date|
+------+-------+------+----------+
|     1|      1|   4.0|2000-07-30|
|     1|      3|   4.0|2000-07-30|
|     1|      6|   4.0|2000-07-30|
|     1|     47|   5.0|2000-07-30|
|     1|     50|   5.0|2000-07-30|
+------+-------+------+----------+
only showing top 5 rows



In [20]:
tagsDF = tagsDF.withColumn("timestamp", tagsDF["timestamp"].cast(TimestampType()).cast(DateType()))\
               .withColumnRenamed("timestamp","date")

In [21]:
tagsDF.show(5)

+------+-------+---------------+----------+
|userId|movieId|            tag|      date|
+------+-------+---------------+----------+
|     2|  60756|          funny|2015-10-24|
|     2|  60756|Highly quotable|2015-10-24|
|     2|  60756|   will ferrell|2015-10-24|
|     2|  89774|   Boxing story|2015-10-24|
|     2|  89774|            MMA|2015-10-24|
+------+-------+---------------+----------+
only showing top 5 rows



Double check that the **date** column spans over the time frame specified in the disclaimer:

In [22]:
ratingsDF.select(F.min("date").alias("ratings_from_date"),\
                 F.max("date").alias("ratings_to_date")).show()

+-----------------+---------------+
|ratings_from_date|ratings_to_date|
+-----------------+---------------+
|       1996-03-29|     2018-09-24|
+-----------------+---------------+



In [23]:
tagsDF.select(F.min("date").alias("tags_from_date"),\
              F.max("date").alias("tags_to_date")).show()

+--------------+------------+
|tags_from_date|tags_to_date|
+--------------+------------+
|    2006-01-13|  2018-09-16|
+--------------+------------+



### Inspection of rating frequency

Show the aggregated number of ratings per year, including a proxy for graphical representation:

In [24]:
print("Average number of ratings per year: {}".format(round(ratingsDF.count()/23, 2)))

ratingsDF.groupBy(F.year(F.col("date")).alias("year"))\
         .count()\
         .orderBy(F.col("year").asc())\
         .withColumn("count_bar",(F.col("count")/100).cast(IntegerType()))\
         .withColumn("count_bar",F.expr("repeat('|',count_bar)"))\
         .show(23,False)

Average number of ratings per year: 4384.17
+----+-----+----------------------------------------------------------------------------------------------------+
|year|count|count_bar                                                                                           |
+----+-----+----------------------------------------------------------------------------------------------------+
|1996|6040 |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||                                        |
|1997|1916 ||||||||||||||||||||                                                                                 |
|1998|507  ||||||                                                                                               |
|1999|2439 |||||||||||||||||||||||||                                                                            |
|2000|10061||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
|2001|3922 |||||||||||||||||||||||||||||||||

The years 2000, 2017 and 2007 comprise the largest amount of ratings over all users. Since 2015 the annual rating number has been consistently large compared to its historical scale.

Show the average monthly number of ratings:

In [25]:
ratingsDF.groupBy(F.year(F.col("date")),F.month(F.col("date")).alias("month"))\
         .count()\
         .groupBy(F.col("month"))\
         .agg(F.round(F.mean(F.col("count")),2).alias("avg_number_ratings"))\
         .orderBy(F.col("avg_number_ratings").desc())\
         .show(12)

+-----+------------------+
|month|avg_number_ratings|
+-----+------------------+
|    5|            473.13|
|   11|            439.77|
|    8|            412.41|
|    1|            391.86|
|    9|            386.82|
|    6|            383.83|
|    3|            382.35|
|    4|            371.95|
|    2|            349.91|
|   10|            324.95|
|   12|            311.14|
|    7|            302.17|
+-----+------------------+



May and November have been on average the months with historically more rating activity, which is somewhat surprising as they precede the typical holiday months.

Show the rating levels distribution:

In [26]:
ratingsDF.groupBy(F.col("rating"))\
         .count()\
         .orderBy(F.col("rating").desc())\
         .withColumn("count_bar",(F.col("count")/300).cast(IntegerType()))\
         .withColumn("count_bar",F.expr("repeat('|',count_bar)"))\
         .show(10,False)

+------+-----+-----------------------------------------------------------------------------------------+
|rating|count|count_bar                                                                                |
+------+-----+-----------------------------------------------------------------------------------------+
|5.0   |13211|||||||||||||||||||||||||||||||||||||||||||||                                             |
|4.5   |8551 |||||||||||||||||||||||||||||                                                             |
|4.0   |26818|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
|3.5   |13136||||||||||||||||||||||||||||||||||||||||||||                                              |
|3.0   |20047|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||                       |
|2.5   |5550 |||||||||||||||||||                                                                       |
|2.0   |7551 ||||||||||||||||||||||||||                

4.0 and 3.0 are significantly more frequent than any other rating level. The distribution also shows that users are more prone to giving ratings above 3.0.

## Data exploration

All movies listed in moviesDF have been reviewed or tagged (or both) at least once. The majority of movies (8170) have a rating but not a tag, some (1554) have both a rating and a tag, and a small minority (18) have a tag but not a rating.

Show and explore the 18 movies that are tagged but not rated:

In [27]:
movies_ratingsDF = moviesDF.join(ratingsDF, "movieId", "left_outer")

In [28]:
tagged_unrated_movies = movies_ratingsDF.select(F.col("movieId"))\
                                        .where(F.col("userId").isNull())

tagged_unrated_movies = [row.movieId for row in tagged_unrated_movies.collect()]

In [29]:
movies_ratingsDF.select("title","genres")\
                .where(F.col("movieId").isin(tagged_unrated_movies))\
                .withColumn("releaseYear",F.substring("title",-5,4).cast(IntegerType()))\
                .show(18,False)

+--------------------------------------------+------------------------+-----------+
|title                                       |genres                  |releaseYear|
+--------------------------------------------+------------------------+-----------+
|Innocents, The (1961)                       |Drama|Horror|Thriller   |1961       |
|Niagara (1953)                              |Drama|Thriller          |1953       |
|For All Mankind (1989)                      |Documentary             |1989       |
|Color of Paradise, The (Rang-e khoda) (1999)|Drama                   |1999       |
|I Know Where I'm Going! (1945)              |Drama|Romance|War       |1945       |
|Chosen, The (1981)                          |Drama                   |1981       |
|Road Home, The (Wo de fu qin mu qin) (1999) |Drama|Romance           |1999       |
|Scrooge (1970)                              |Drama|Fantasy|Musical   |1970       |
|Proof (1991)                                |Comedy|Drama|Romance    |1991 

These movies have not (yet) been rated by any user, and hence it can be problematic to resort to them for recommendation purposes.

Explore the movies that have rating but no tag:

In [30]:
movies_tagsDF = moviesDF.join(tagsDF, "movieId", "left_outer")

In [31]:
rated_untagged_movies = movies_tagsDF.select(F.col("movieId"))\
                                     .where(F.col("userId").isNull())

rated_untagged_movies = [row.movieId for row in rated_untagged_movies.collect()]

In [32]:
rated_untaggedDF = movies_ratingsDF.select("title","genres","rating")\
                                   .where(F.col("movieId").isin(rated_untagged_movies))\
                                   .groupBy(F.col("title"))\
                                   .agg(F.round(F.mean(F.col("rating")),2).alias("avg_rating"),
                                        F.count("*").alias("count_ratings"))\
                                   .where("count_ratings > 30")

print("Only {0} out of {1} rated untagged movies have more than 30 reviews".format(rated_untaggedDF.count(),
                                                                                   len(rated_untagged_movies)))

Only 360 out of 8170 rated untagged movies have more than 30 reviews


Focusing on the rated untagged movies *with more than 30 user ratings*, show the top 10 movies in terms of average rating and number of ratings:

In [33]:
rated_untaggedDF.orderBy(F.col("avg_rating").desc())\
                .show(10,truncate=False)

rated_untaggedDF.orderBy(F.col("count_ratings").desc())\
                .show(10,truncate=False)

+----------------------------------------------------------+----------+-------------+
|title                                                     |avg_rating|count_ratings|
+----------------------------------------------------------+----------+-------------+
|Boondock Saints, The (2000)                               |4.22      |43           |
|Brazil (1985)                                             |4.18      |59           |
|Cinema Paradiso (Nuovo cinema Paradiso) (1989)            |4.16      |34           |
|Snatch (2000)                                             |4.16      |93           |
|For a Few Dollars More (Per qualche dollaro in più) (1965)|4.15      |33           |
|Lives of Others, The (Das leben der Anderen) (2006)       |4.12      |34           |
|Toy Story 3 (2010)                                        |4.11      |55           |
|Boogie Nights (1997)                                      |4.08      |39           |
|American Beauty (1999)                               

More succinctly, the average rating and rating count of the set of movies above spans over the following intervals:

In [34]:
rated_untaggedDF.select("avg_rating","count_ratings").groupBy()\
                .agg(F.max("avg_rating"),F.min("avg_rating"),
                     F.max("count_ratings"),F.min("count_ratings"))\
                .show()

+---------------+---------------+------------------+------------------+
|max(avg_rating)|min(avg_rating)|max(count_ratings)|min(count_ratings)|
+---------------+---------------+------------------+------------------+
|           4.22|           1.95|               204|                31|
+---------------+---------------+------------------+------------------+



Tag exploration. What is the average number of tags per movie in tagsDF? And the average number of tags per user? How does it compare with the average number of tags a user assigns to a movie?

In [35]:
tagsDF.groupBy(F.col("movieId"))\
      .count()\
      .agg(F.round(F.mean(F.col("count")),1).alias("avg_tags_per_movie"))\
      .show()

tagsDF.groupBy(F.col("userId"))\
      .count()\
      .agg(F.mean(F.col("count")).alias("avg_tags_per_user"))\
      .show()

tagsDF.groupBy(F.col("userId"),F.col("movieId"))\
      .count()\
      .agg(F.round(F.mean(F.col("count")),1).alias("avg_tags_per_user_per_movie"))\
      .show()

+------------------+
|avg_tags_per_movie|
+------------------+
|               2.3|
+------------------+

+-----------------+
|avg_tags_per_user|
+-----------------+
|             63.5|
+-----------------+

+---------------------------+
|avg_tags_per_user_per_movie|
+---------------------------+
|                        2.1|
+---------------------------+



Identify the users that tagged movies without rating them:

In [36]:
users_tag_notrate = tagsDF.select(F.col("userId"))\
                          .where(F.col("movieId").isin(tagged_unrated_movies))

users_tag_notrate = [row.userId for row in users_tag_notrate.collect()]

In [37]:
print("Movies with tags but without rating:")

tagsDF.select("*")\
      .where(F.col("movieId").isin(tagged_unrated_movies))\
      .where(F.col("userId").isin(users_tag_notrate))\
      .agg(F.countDistinct("userId").alias("distinct_users"),
           F.countDistinct("movieId").alias("distinct_movies"),
           F.count(F.lit(1)).alias("total_tag_count"))\
      .show()

movies_tagsDF.select("userId","title","tag","date")\
             .where(F.col("movieId").isin(tagged_unrated_movies))\
             .where(F.col("userId").isin(users_tag_notrate))\
             .orderBy(F.col("userId").desc())\
             .show(21,False)

Movies with tags but without rating:
+--------------+---------------+---------------+
|distinct_users|distinct_movies|total_tag_count|
+--------------+---------------+---------------+
|             4|             18|             21|
+--------------+---------------+---------------+

+------+--------------------------------------------+--------------------------+----------+
|userId|title                                       |tag                       |date      |
+------+--------------------------------------------+--------------------------+----------+
|543   |Chalet Girl (2011)                          |Comedy                    |2013-08-20|
|474   |Niagara (1953)                              |In Netflix queue          |2006-01-14|
|474   |Innocents, The (1961)                       |governess                 |2006-01-17|
|474   |I Know Where I'm Going! (1945)              |In Netflix queue          |2006-01-14|
|474   |Chosen, The (1981)                          |Judaism             

About 2/3 of the tags above are not useful to infer how these users may rate these movies, as they relate to other aspects like queue status or movie availability.

Rating exploration. What is the average number of ratings per user in ratings DF? And the average number of ratings per movie?

In [38]:
ratingsDF.groupBy(F.col("userId"))\
         .count()\
         .agg(F.round(F.mean(F.col("count")),1).alias("avg_ratings_per_user"))\
         .show()

ratingsDF.groupBy(F.col("movieId"))\
         .count()\
         .agg(F.round(F.mean(F.col("count")),1).alias("avg_ratings_per_movie"))\
         .show()

+--------------------+
|avg_ratings_per_user|
+--------------------+
|               165.3|
+--------------------+

+---------------------+
|avg_ratings_per_movie|
+---------------------+
|                 10.4|
+---------------------+



## Basic investigation on service oriented business questions

### What is the predominant (frequency based) genre per rating level?

In [39]:
rating_maxcountDF = movies_ratingsDF.where("userId is not null")\
                                    .groupBy(F.col("rating"),F.col("genres"))\
                                    .count()\
                                    .groupBy("rating").agg(F.max(F.col("count")).alias("count"))
                                            
rating_maxcountDF.join(movies_ratingsDF.where("userId is not null")\
                                       .groupBy(F.col("rating"),F.col("genres"))\
                                       .count(),
                       ["count","rating"])\
                 .withColumnRenamed("genres","predominant_genre")\
                 .orderBy(F.col("rating").desc())\
                 .show()

+-----+------+-----------------+
|count|rating|predominant_genre|
+-----+------+-----------------+
|  895|   5.0|            Drama|
|  593|   4.5|            Drama|
| 2055|   4.0|            Drama|
|  854|   3.5|           Comedy|
| 1614|   3.0|           Comedy|
|  515|   2.5|           Comedy|
|  828|   2.0|           Comedy|
|  256|   1.5|           Comedy|
|  348|   1.0|           Comedy|
|  136|   0.5|           Comedy|
+-----+------+-----------------+



The classic genres (pure drama/comedy) account for the most frequent ratings on every level. This is partly expected because Drama and Comedy (out of the 951 distinct genre combinations) account for roughly 20% of all movies in moviesDF:

In [40]:
moviesDF.groupBy(F.col("genres")).count()\
        .withColumn("proportion(%)",F.round(F.col("count")/(moviesDF.count())*100,3))\
        .orderBy(F.col("count").desc())\
        .show(10,False)

+--------------------+-----+-------------+
|genres              |count|proportion(%)|
+--------------------+-----+-------------+
|Drama               |1053 |10.809       |
|Comedy              |946  |9.711        |
|Comedy|Drama        |435  |4.465        |
|Comedy|Romance      |363  |3.726        |
|Drama|Romance       |349  |3.582        |
|Documentary         |339  |3.48         |
|Comedy|Drama|Romance|276  |2.833        |
|Drama|Thriller      |168  |1.724        |
|Horror              |167  |1.714        |
|Horror|Thriller     |135  |1.386        |
+--------------------+-----+-------------+
only showing top 10 rows



This is further illustrated below, where the top 10 most/least rated genres are shown:

In [41]:
movies_ratingsDF.groupBy("genres")\
                .agg(F.count("rating").alias("count_ratings"))\
                .orderBy(F.col("count_ratings").desc())\
                .show(10,False)

movies_ratingsDF.groupBy("genres")\
                .agg(F.count("rating").alias("count_ratings"))\
                .orderBy(F.col("count_ratings").asc())\
                .show(10,False)

+-------------------------+-------------+
|genres                   |count_ratings|
+-------------------------+-------------+
|Comedy                   |7196         |
|Drama                    |6291         |
|Comedy|Romance           |3967         |
|Comedy|Drama|Romance     |3000         |
|Comedy|Drama             |2851         |
|Drama|Romance            |2838         |
|Action|Adventure|Sci-Fi  |2361         |
|Crime|Drama              |2315         |
|Action|Crime|Thriller    |1554         |
|Action|Adventure|Thriller|1455         |
+-------------------------+-------------+
only showing top 10 rows

+------------------------------------------------------+-------------+
|genres                                                |count_ratings|
+------------------------------------------------------+-------------+
|Action|Animation|Comedy|Fantasy                       |1            |
|Comedy|Drama|Fantasy|Mystery|Romance                  |1            |
|Documentary|Drama|Thriller    

Unconventional genre combinations (e.g. horror|romance|sci-fi) barely have any ratings, as such movies are uncommon and typically not compelling to a wide audience. In comparison, what are the top 10 genres in terms of average rating (with more than 30 ratings for a genre to be considered)?

In [42]:
movies_ratingsDF.groupBy("genres")\
                .agg(F.round(F.mean("rating"),2).alias("avg_rating"),
                     F.count("rating").alias("count_ratings"))\
                .where("count_ratings > 30")\
                .orderBy(F.col("avg_rating").desc())\
                .show(10,False)

+-----------------------------------------+----------+-------------+
|genres                                   |avg_rating|count_ratings|
+-----------------------------------------+----------+-------------+
|Action|Crime|Drama|IMAX                  |4.23      |151          |
|Action|Adventure|Comedy|Fantasy|Romance  |4.23      |144          |
|Drama|Film-Noir|Romance                  |4.18      |33           |
|Action|Adventure|Mystery|Romance|Thriller|4.18      |57           |
|Film-Noir|Mystery                        |4.17      |44           |
|Fantasy|Sci-Fi                           |4.16      |61           |
|Comedy|Drama|Romance|War                 |4.16      |421          |
|Action|Adventure|Crime|Drama|Thriller    |4.15      |75           |
|Action|Adventure|Western                 |4.14      |73           |
|Comedy|Crime|Drama|Thriller              |4.13      |563          |
+-----------------------------------------+----------+-------------+
only showing top 10 rows



The genres with highest average rating are combined rather than 'pure'. *Drama*, *Action* and *Comedy* appear in a notable proportion of the combinations.

### What is the predominant tag per genre and the most tagged genres?

In [43]:
tag_maxcountDF = movies_tagsDF.where("userId is not null")\
                              .groupBy(F.col("tag"),F.col("genres"))\
                              .count()\
                              .groupBy("genres").agg(F.max(F.col("count")).alias("count"))
                                            
tag_maxcountDF.join(movies_tagsDF.where("userId is not null")\
                                 .groupBy(F.col("tag"),F.col("genres"))\
                                 .count(),
                    ["count","genres"])\
              .orderBy(F.col("count").desc())\
              .show(truncate=False)

+-----+----------------------------+------------------+
|count|genres                      |tag               |
+-----+----------------------------+------------------+
|36   |Drama                       |In Netflix queue  |
|14   |Documentary                 |In Netflix queue  |
|8    |Drama|Romance               |In Netflix queue  |
|7    |Comedy                      |In Netflix queue  |
|7    |Comedy                      |comedy            |
|6    |Comedy|Musical|Romance      |Astaire and Rogers|
|6    |Action|Adventure|Sci-Fi     |superhero         |
|6    |Crime|Drama                 |Mafia             |
|6    |Comedy|Drama                |In Netflix queue  |
|5    |Horror                      |Jason             |
|5    |Drama|War                   |Vietnam           |
|5    |Comedy|Drama|Romance        |In Netflix queue  |
|4    |Action|Sci-Fi|Thriller      |sci-fi            |
|4    |Comedy|Romance              |Hepburn and Tracy |
|4    |Horror|Thriller             |ghosts      

About half of the most frequent tags are actually platform related, as they refer to the corresponding movie queue status instead of the movie contents.

### What are the most predominant (popularity based) movies?

Top 10 movies in terms of **number of ratings**:

In [44]:
movies_ratingsDF.groupBy("title")\
                .agg(F.count("rating").alias("count_ratings"),
                     F.round(F.mean("rating"),3).alias("avg_rating"))\
                .orderBy(F.col("count_ratings").desc())\
                .show(10,False)

+-----------------------------------------+-------------+----------+
|title                                    |count_ratings|avg_rating|
+-----------------------------------------+-------------+----------+
|Forrest Gump (1994)                      |329          |4.164     |
|Shawshank Redemption, The (1994)         |317          |4.429     |
|Pulp Fiction (1994)                      |307          |4.197     |
|Silence of the Lambs, The (1991)         |279          |4.161     |
|Matrix, The (1999)                       |278          |4.192     |
|Star Wars: Episode IV - A New Hope (1977)|251          |4.231     |
|Jurassic Park (1993)                     |238          |3.75      |
|Braveheart (1995)                        |237          |4.032     |
|Terminator 2: Judgment Day (1991)        |224          |3.971     |
|Schindler's List (1993)                  |220          |4.225     |
+-----------------------------------------+-------------+----------+
only showing top 10 rows



Top 10 movies in terms of **average rating** (*provided more than 30 users reviewed them*):

In [45]:
movies_ratingsDF.groupBy("title")\
                .agg(F.count("rating").alias("count_ratings"),
                     F.round(F.mean("rating"),3).alias("avg_rating"))\
                .where("count_ratings > 30")\
                .orderBy(F.col("avg_rating").desc())\
                .show(10,False)

+---------------------------------------------------------------------------+-------------+----------+
|title                                                                      |count_ratings|avg_rating|
+---------------------------------------------------------------------------+-------------+----------+
|Shawshank Redemption, The (1994)                                           |317          |4.429     |
|Lawrence of Arabia (1962)                                                  |45           |4.3       |
|Godfather, The (1972)                                                      |192          |4.289     |
|Fight Club (1999)                                                          |218          |4.273     |
|Cool Hand Luke (1967)                                                      |57           |4.272     |
|Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)|97           |4.268     |
|Rear Window (1954)                                                      

Interestingly, the above tables only have one movie in common (Shawshank Redemption), which suggests that some distinction must be made between movies that are generally liked and movies that are widely watched.

### Do popular movies belong to a particular time frame?

Taking as reference the tables above, 9/10 of the top 10 movies by **number of ratings** are from the 1990s.

In the table of the top 10 movies by **average rating**, the distribution is:

* 3/10 from the 1990s
* 3/10 from the 1960s
* 2/10 from the 1970s
* 1/10 from the 1950s
* 1/10 from the 2000s

Show a distribution of the database movies per release decade, and see how it compares with this:

In [46]:
moviesDF.select("title","genres")\
        .withColumn("releaseYear",F.substring(F.rtrim(F.col("title")),-5,4).cast(IntegerType()))\
        .groupBy().agg(F.min("releaseYear"),
                       F.max("releaseYear"))\
        .show()

moviesDF.select("title","genres")\
        .withColumn("releaseYear",F.substring(F.rtrim(F.col("title")),-5,4).cast(IntegerType()))\
        .select(F.col("title"),F.col("releaseYear"))\
        .where("releaseYear is null")\
        .collect()

+----------------+----------------+
|min(releaseYear)|max(releaseYear)|
+----------------+----------------+
|            1902|            2018|
+----------------+----------------+



[Row(title='"11\'09""01 - September 11 (2002)"', releaseYear=None),
 Row(title='Babylon 5', releaseYear=None),
 Row(title='Ready Player One', releaseYear=None),
 Row(title='Hyena Road', releaseYear=None),
 Row(title='The Adventures of Sherlock Holmes and Doctor Watson', releaseYear=None),
 Row(title='Nocturnal Animals', releaseYear=None),
 Row(title='Paterson', releaseYear=None),
 Row(title='Moonlight', releaseYear=None),
 Row(title='The OA', releaseYear=None),
 Row(title='Cosmos', releaseYear=None),
 Row(title='Maria Bamford: Old Baby', releaseYear=None),
 Row(title='Generation Iron 2', releaseYear=None),
 Row(title='Black Mirror', releaseYear=None)]

The 13 movie titles above do not have the same format as the rest (as warned in the disclaimer) and extracting their release year is more involved. They will be disregarded for the time being.

Create a new column showing the release decade and aggregate by counting per decade:

In [47]:
moviesDF.select("title","genres")\
        .withColumn("releaseYear",F.substring(F.rtrim(F.col("title")),-5,4).cast(IntegerType()))\
        .withColumn("releaseDecade", F.when((F.col("releaseYear") >= 1900) & (F.col("releaseYear") < 1910), "1900s")\
                                      .when((F.col("releaseYear") >= 1910) & (F.col("releaseYear") < 1920), "1910s")\
                                      .when((F.col("releaseYear") >= 1920) & (F.col("releaseYear") < 1930), "1920s")\
                                      .when((F.col("releaseYear") >= 1930) & (F.col("releaseYear") < 1940), "1930s")\
                                      .when((F.col("releaseYear") >= 1940) & (F.col("releaseYear") < 1950), "1940s")\
                                      .when((F.col("releaseYear") >= 1950) & (F.col("releaseYear") < 1960), "1950s")\
                                      .when((F.col("releaseYear") >= 1960) & (F.col("releaseYear") < 1970), "1960s")\
                                      .when((F.col("releaseYear") >= 1970) & (F.col("releaseYear") < 1980), "1970s")\
                                      .when((F.col("releaseYear") >= 1980) & (F.col("releaseYear") < 1990), "1980s")\
                                      .when((F.col("releaseYear") >= 1990) & (F.col("releaseYear") < 2000), "1990s")\
                                      .when((F.col("releaseYear") >= 2000) & (F.col("releaseYear") < 2010), "2000s")\
                                      .when((F.col("releaseYear") >= 2010) & (F.col("releaseYear") < 2020), "2010s"))\
        .where("releaseYear is not null")\
        .groupBy(F.col("releaseDecade"))\
        .count().withColumnRenamed("count","movie_count")\
        .withColumn("proportion(%)", F.round(100 * F.col("movie_count")/(moviesDF.count()-13), 2))\
        .orderBy(F.col("releaseDecade").asc())\
        .withColumn("count_bar",(F.col("movie_count")/40).cast(IntegerType()))\
        .withColumn("count_bar",F.expr("repeat('|',count_bar)"))\
        .show(truncate=False)

+-------------+-----------+-------------+-----------------------------------------------------------------------+
|releaseDecade|movie_count|proportion(%)|count_bar                                                              |
+-------------+-----------+-------------+-----------------------------------------------------------------------+
|1900s        |3          |0.03         |                                                                       |
|1910s        |7          |0.07         |                                                                       |
|1920s        |37         |0.38         |                                                                       |
|1930s        |136        |1.4          ||||                                                                    |
|1940s        |197        |2.02         |||||                                                                   |
|1950s        |279        |2.87         |||||||                                         

Most movies were released in the 2000s or the 1990s, though in fairness the 2010s decade is not completely represented in this dataset. It is noteworthy that movies from the 1950s-1970s are amongst the most popular (highly rated on average) despite comprising significantly fewer titles than more recent times. Put into perspective, though, 50 or 60 years ago the cinematographic industry could afford to produce less movies per year than nowadays, and the ones persisting in time (hence being incorporated into online platform catalogues) are largely classics prone to attract the audience.

### Training a recommendation engine to predict ratings

In [48]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="nan", nonnegative=False)

recommender = als.fit(ratingsDF.select("userId","movieId","rating"))

Could the recommendation engine be used to predict the rating of tagged unrated movies?

In [49]:
users_tag_notrateDF = tagsDF.select(F.col("userId"),F.col("movieId"))\
                            .where(F.col("movieId").isin(tagged_unrated_movies))

rate_predictions = recommender.transform(users_tag_notrateDF)
rate_predictions.show()

+------+-------+----------+
|userId|movieId|prediction|
+------+-------+----------+
|   474|   3456|       NaN|
|   474|   2939|       NaN|
|   288|   7020|       NaN|
|   474|   6668|       NaN|
|   474|   7792|       NaN|
|   474|  26085|       NaN|
|   474|  34482|       NaN|
|   474|  32160|       NaN|
|   318|  30892|       NaN|
|   318|  30892|       NaN|
|   318|  30892|       NaN|
|   474|   6849|       NaN|
|   474|   1076|       NaN|
|   543|  85565|       NaN|
|   474|   3338|       NaN|
|   474|  25855|       NaN|
|   474|   4194|       NaN|
|   474|   8765|       NaN|
|   474|   5721|       NaN|
|   474|   5721|       NaN|
+------+-------+----------+
only showing top 20 rows



None of these movies has been watched by anyone, hence the NaN values (consistently with the *coldStartStrategy* parameter). Focus must be shifted to predict users' ratings of movies that have indeed been watched by others.

Show predictions for the top 10 users in terms of **number of ratings**:

In [50]:
total_avg_rating = ratingsDF.groupBy().agg(F.mean(F.col("rating")).alias("total_avg_rating"))\
                            .collect()[0]["total_avg_rating"]
print("The overall average rating amongst all users is {0:.3f}".format(total_avg_rating))

top10usersDF = ratingsDF.groupBy(F.col("userId"))\
                        .agg(F.count(F.col("rating")).alias("count_ratings"),
                             F.round(F.mean(F.col("rating")),3).alias("avg_rating"))\
                        .orderBy(F.col("count_ratings").desc())\
                        .limit(10)
top10usersDF.show()

The overall average rating amongst all users is 3.502
+------+-------------+----------+
|userId|count_ratings|avg_rating|
+------+-------------+----------+
|   414|         2698|     3.392|
|   599|         2478|     2.642|
|   474|         2108|     3.399|
|   448|         1864|     2.847|
|   274|         1346|     3.236|
|   610|         1302|     3.689|
|    68|         1260|     3.234|
|   380|         1218|     3.673|
|   606|         1115|     3.657|
|   288|         1055|     3.146|
+------+-------------+----------+



After training the recommender with the entire dataset, retrieve the top recommendation for each user amongst the top 10 above:

In [51]:
rec_for_top10users = recommender.recommendForAllUsers(1).join(top10usersDF,"userId")

rec_for_top10users = rec_for_top10users.withColumn("recommendations", F.col("recommendations").getItem(0))\
                                       .withColumn("pred_rating",F.col("recommendations")["rating"])\
                                       .withColumn("recommendations",F.col("recommendations")["movieId"])\
                                       .withColumnRenamed("recommendations","movieId")

rec_for_top10users = rec_for_top10users.join(moviesDF.select("movieId","title"),"movieId")\
                                       .orderBy(F.col("count_ratings").desc())\
                                       .withColumnRenamed("title","recommended_title")

rec_for_top10users.show(truncate=False)

+-------+------+-------------+----------+-----------+-----------------------------------------------------+
|movieId|userId|count_ratings|avg_rating|pred_rating|recommended_title                                    |
+-------+------+-------------+----------+-----------+-----------------------------------------------------+
|3266   |414   |2698         |3.392     |6.0033336  |Man Bites Dog (C'est arrivé près de chez vous) (1992)|
|6461   |599   |2478         |2.642     |4.8107777  |Unforgiven, The (1960)                               |
|3265   |474   |2108         |3.399     |5.928932   |Hard-Boiled (Lat sau san taam) (1992)                |
|501    |448   |1864         |2.847     |5.8097444  |Naked (1993)                                         |
|3200   |274   |1346         |3.236     |5.687201   |Last Detail, The (1973)                              |
|3508   |610   |1302         |3.689     |5.5455465  |Outlaw Josey Wales, The (1976)                       |
|2459   |68    |1260        

The rating is considered a continuous variable, hence the predictions may go beyond the theoretical upper bound of 5.0. For all ten users, the predicted rating is considerably above their historical average.

Additionally, let's consider if the recommended movies are consistent with the historical record of these users in terms of movie release year:

In [52]:
top10users = rec_for_top10users.select("userId").collect()
top10users = [row.userId for row in top10users]

movies_ratingsDF.where(F.col("userId").isin(top10users))\
                .withColumn("releaseYear",F.substring(F.rtrim(F.col("title")),-5,4).cast(IntegerType()))\
                .groupBy("userId")\
                .agg(F.round(F.mean(F.col("releaseYear")),0).cast(IntegerType()).alias("avg_movieYear"),
                     F.min(F.col("releaseYear")).alias("min_movieYear"),
                     F.max(F.col("releaseYear")).alias("max_movieYear"))\
                .join(rec_for_top10users.select("userId","recommended_title"),"userId")\
                .withColumn("recommended_title",
                            F.substring(F.rtrim(F.col("recommended_title")),-5,4).cast(IntegerType()))\
                .withColumnRenamed("recommended_title","recommended_movieYear")\
                .show()

+------+-------------+-------------+-------------+---------------------+
|userId|avg_movieYear|min_movieYear|max_movieYear|recommended_movieYear|
+------+-------------+-------------+-------------+---------------------+
|   606|         1988|         1921|         2012|                 1982|
|   599|         1994|         1902|         2017|                 1960|
|   274|         1997|         1932|         2011|                 1973|
|   474|         1983|         1921|         2008|                 1992|
|   610|         2003|         1939|         2017|                 1976|
|   380|         1999|         1922|         2018|                 1995|
|   448|         2000|         1930|         2017|                 1993|
|   288|         1989|         1933|         2009|                 1997|
|   414|         1996|         1927|         2018|                 1992|
|    68|         1998|         1936|         2017|                 1974|
+------+-------------+-------------+-------------+-

With the exception of users 474 and 288 (whereby interestingly they have the lowest movieYear average), all other users are suggested to watch older movies than the average they rate. This is consistent with the previous finding that there are notably less movies from past decades but they are more likely to be appreciated by active users. This is also reinforced by the fact that all users have been exposed to older movies (from the 30s or earlier) before.